In [1]:
import findspark
findspark.init()
import pyspark
from IPython.display import display, clear_output
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()


In [17]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate();

jsonString = {
    "id": 900,
    "gender": 0,
    "age": 0.08,
    "hypertension": 0,
    "heart_disease": 0,
    "ever_married": 0,
    "work_type": 1,
    "Residence_type": 1,
    "avg_glucose_level": 139.67,
    "bmi": 14.1,
    "smoking_status": 0
}

df = spark.read.json(sc.parallelize([jsonString]))
schema = df.schema

In [18]:
lines = spark\
        .readStream\
        .schema(schema)\
        .format('socket')\
        .option('host', "localhost")\
        .option('port', 9999)\
        .load()

Py4JJavaError: An error occurred while calling o134.load.
: java.lang.UnsupportedOperationException: TextSocketSourceProvider source does not support user-specified schema.
	at org.apache.spark.sql.execution.datasources.v2.DataSourceV2Utils$.getTableFromProvider(DataSourceV2Utils.scala:76)
	at org.apache.spark.sql.streaming.DataStreamReader.load(DataStreamReader.scala:216)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.Gate

In [11]:
query = words\
    .writeStream\
    .queryName("wordCounts")\
    .outputMode('complete')\
    .format('memory')\
    .start()

TypeError: 'Column' object is not callable

In [9]:
display(spark.sql(f"SELECT * FROM {query.name}").show())

+-------+-----+
|   word|count|
+-------+-----+
|    yay|    1|
|    man|    1|
|     is|    1|
|     on|    1|
|  going|    1|
|    the|    1|
|dumbass|    1|
|   what|    1|
|fucking|    1|
+-------+-----+



None